## 什么是优化器

[https://www.youtube.com/watch?v=Ym1m4Bh14ps](https://www.youtube.com/watch?v=Ym1m4Bh14ps)

* 用来寻找函数最小值
* 从数据中查找并建立参数化模型（从实验数据中找到一个多项式来拟合实验数据）
* 使用优化器来改善股票投资组合中的分配方式

## 凸优化

[https://zh.wikipedia.org/wiki/%E5%87%B8%E5%84%AA%E5%8C%96](https://zh.wikipedia.org/wiki/%E5%87%B8%E5%84%AA%E5%8C%96)